# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( 'data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
#Python 3 print(child.find('name').text)" instead of "print child.find('name').text
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import pandas as pd
import numpy as np

In [6]:
document = ET.parse( './data/mondial_database.xml' )

In [7]:
root = document.getroot()
country_list = [country for country in root.findall('country')]
mortality_list = []
for country in country_list:
    name = country.findtext('name')
    mortality = country.findtext('infant_mortality')
    if mortality == None:
        mortality = '0'
    mortality_list.append((name, float(mortality)))

sorted(mortality_list, key=lambda x:x[1], reverse=True)[0:10]


[('Western Sahara', 145.82),
 ('Afghanistan', 117.23),
 ('Mali', 104.34),
 ('Somalia', 100.14),
 ('Central African Republic', 92.86),
 ('Guinea-Bissau', 90.92),
 ('Chad', 90.3),
 ('Niger', 86.27),
 ('Angola', 79.99),
 ('Burkina Faso', 76.8)]

In [8]:
citypop_list = []
for country in country_list:
    city_list = [city for city in country.findall('city')]
    for city in city_list:
        citypop = city.findtext('population')
        if citypop == None:
            citypop = '0'
        citypop_list.append((city.findtext('name'), int(citypop)))
        
sorted(citypop_list, key=lambda x:x[1], reverse=True)[0:10]

[('Seoul', 10229262),
 ('Hong Kong', 7055071),
 ('Al Qahirah', 6053000),
 ('Bangkok', 5876000),
 ('Ho Chi Minh', 3924435),
 ('Busan', 3813814),
 ('New Taipei', 3722082),
 ('Hanoi', 3056146),
 ('Al Iskandariyah', 2917000),
 ('Taipei', 2626138)]

#question 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [9]:
ethnicgps=[]
allpops=[]
# 
for elem in document.iterfind('country'):
    country = elem.find('name').text
    pop=elem.find('population')

#This will find Ethnic group and % of Ethnic group in a given country
    for e in elem.iterfind('ethnicgroup'):
        group= e.text
        percents = float(e.attrib['percentage'])

# Country, ethnic group and its percentage saved in ethnicgps list        
        ethnicgps.append([country,group,percents]) 
    
# This will return year and population present in a given country
    for y in elem.iterfind('population'):
        year=int(y.attrib['year'])
        pop=int(y.text)

# country name, year and population saved in allpops list
        allpops.append([country, year, pop])


#convert list to dataframe
ethnicdf=pd.DataFrame(ethnicgps, columns=['Country', 'Ethnic Group', 'Percentage'])

#allpops list converted to dataFrame popdf    
popdf=pd.DataFrame(allpops, columns=['Country', 'Year', 'Population'])  

#only the latest year on each country are extracted from popdf
idx=popdf.groupby(['Country'])['Year'].transform(max)==popdf['Year']
popdf=popdf[idx]

#popdf and ethnicdf dataframe are merged on entries present on both. 
#Country columns present on both used to merge both DFs
Etpop=popdf.merge(ethnicdf, how='inner')

#Ethnicity percentage multiplied by population of the given country and saved in "Ethnic Group Population" column
Etpop['Ethnic Group Population']=Etpop.Percentage/100*Etpop.Population

#Sum of Each ethnic group is calculated and sorted from highest to lowest and returns only top 10 highest
Etpop.groupby(['Ethnic Group'])['Ethnic Group Population'].sum().sort_values(ascending=False).head(10)

Ethnic Group
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: Ethnic Group Population, dtype: float64

4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

4a: Longest River

In [10]:
country_code=[]
for elem in document.iterfind('country'):
    country_name=elem.find('name').text
    code=elem.attrib['car_code']
    country_code.append([country_name, code])
dfcountry_code=pd.DataFrame(country_code, columns=['Country', 'Country_Code'])
dfcountry_code.set_index('Country')

,Country_Code
Country,
Albania,AL
Greece,GR
Macedonia,MK
Serbia,SRB
Montenegro,MNE
Kosovo,KOS
Andorra,AND
France,F
Spain,E


In [11]:
river_list=[]
#list name of river, length of river, and country of river   
for river in document.iterfind('river'):
    river_name=river.find('name').text
    river_length=river.find('length')
    country=river.attrib['country']

# get river length
    if river_length !=None:
        river_length=river.find('length').text

#append country, name, and length to list        
    river_list.append([country, river_name, river_length])
    
#convert list to df
dfriver=pd.DataFrame(river_list, columns=['Country_Code', 'River_Name', 'Length'])

#convert length to numeric
dfriver['Length']=pd.to_numeric(dfriver.Length)

#sort rivers by lengths and print longest one
Longest=dfriver.sort_values(by='Length', ascending=False).head(1)
Longest

,Country_Code,River_Name,Length
174,CO BR PE,Amazonas,6448.0


Largest Lake

In [12]:
lake_list=[]

#This will find lake name area and country location 
for lake in document.iterfind('lake'):
    lake_name=lake.find('name').text
    lake_area=lake.find('area')
    cn=lake.attrib['country']

# This will extract lake area on available entries
    if lake_area !=None:
        lake_area=lake.find('area').text
        
#country name, lake name and lake area saved on lake_list        
    lake_list.append([cn, lake_name, lake_area])

#lake_list converted to DataFrame dflake
dflake=pd.DataFrame(lake_list, columns=['Country_Code', 'Lake_Name', 'Area'])

# Area column converted to numeric 
dflake['Area']=pd.to_numeric(dflake.Area)

#DataFrame sorted from highest ot lowest and returns top 1
lgst=dflake.sort_values(by='Area', ascending=False).head(1)
lgst

,Country_Code,Lake_Name,Area
54,R AZ KAZ IR TM,Caspian Sea,386400.0


4d: Highest Elevation Airport

In [13]:
airport_list=[]
# find airport name, elevation and country code
for airport in document.iterfind('airport'):
    airport_name=airport.find('name').text
    elevation=airport.find('elevation')
    cn=airport.attrib['country']
    
# find elevation
    if elevation !=None:
        elevation=airport.find('elevation').text

#append to list
    airport_list.append([cn, airport_name, elevation])
    
#convert list to df
dfairport=pd.DataFrame(airport_list, columns=['Country_Code', 'Airport_Name', 'Elevation'])

#convert elevation to numeric
dfairport['Elevation']=pd.to_numeric(dfairport.Elevation)

#sort by elevation and print highest elevation
dfairport.sort_values(by='Elevation', ascending=False).head(1).merge(dfcountry_code, how='inner')

,Country_Code,Airport_Name,Elevation,Country
0,BOL,El Alto Intl,4063.0,Bolivia
